In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from bokbokbok.loss_functions.classification import WeightedCrossEntropyLoss
from bokbokbok.eval_metrics.classification import WeightedCrossEntropyMetric

X, y = make_classification(n_samples=1000, 
                           n_features=10, 
                           random_state=41114)

X_train, X_valid, y_train, y_valid = train_test_split(X, 
                                                      y, 
                                                      test_size=0.25, 
                                                      random_state=41114)

### Usage in LightGBM

In [ ]:
import lightgbm as lgb

train = lgb.Dataset(X_train, y_train)
valid = lgb.Dataset(X_valid, y_valid, reference=train)
params_wce = {
     'n_estimators': 300,
     'seed': 41114,
     'n_jobs': 8,
     'learning_rate': 0.1,
   }

wce_clf = lgb.train(params=params_wce,
                train_set=train,
                valid_sets=[train, valid],
                valid_names=['train','valid'],
                fobj=WeightedCrossEntropyLoss(alpha=0.7),
                feval=WeightedCrossEntropyMetric(alpha=0.7),
                early_stopping_rounds=100)

### Usage in XGBoost

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)

params = {
     'seed': 41114,
     'learning_rate': 0.1,
    'disable_default_eval_metric': 1
   }

bst = xgb.train(params,
          dtrain=dtrain,
          num_boost_round=300,
          early_stopping_rounds=10,
          verbose_eval=10,
          obj=WeightedCrossEntropyLoss(alpha=1.0),
          maximize=False,
          feval=WeightedCrossEntropyMetric(alpha=1.0, XGBoost=True),
          evals=[(dtrain, 'dtrain'), (dvalid, 'dvalid')])